In [69]:
## MLFLOW_TRACKING_URI='https://qualiextra-project-6b29e931dd67.herokuapp.com/'

In [70]:
import pandas as pd
import datetime
import numpy as np
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier

In [71]:
data = pd.read_csv("./Missions.csv", sep=';')

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4771 entries, 0 to 4770
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Propriété      4764 non-null   object 
 1   hôtel          4760 non-null   object 
 2   extra          4761 non-null   object 
 3   date           4771 non-null   object 
 4   statuts        617 non-null    object 
 5   tarif urgence  225 non-null    float64
 6   tarif horaire  4760 non-null   float64
 7   nbre d'heures  4771 non-null   object 
 8   total HT       4771 non-null   int64  
 9   facture        3798 non-null   object 
 10  règlement      3743 non-null   object 
 11  montant HT     4771 non-null   int64  
 12  Texte          7 non-null      object 
dtypes: float64(2), int64(2), object(9)
memory usage: 484.7+ KB


In [73]:
mask = ((data["hôtel"].notna()))
data = data[mask]
data.shape

(4760, 13)

In [74]:
data["date_debut"] = data.apply(lambda x: x["date"].split(" →")[0],axis=1)
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC+3)", ""))
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC)", ""))

In [75]:
data["date_debut"] = pd.to_datetime(data["date_debut"], format="%d/%m/%Y %H:%M")

In [76]:
data.dtypes

Propriété                object
hôtel                    object
extra                    object
date                     object
statuts                  object
tarif urgence           float64
tarif horaire           float64
nbre d'heures            object
total HT                  int64
facture                  object
règlement                object
montant HT                int64
Texte                    object
date_debut       datetime64[ns]
dtype: object

In [77]:
data["time_delta"] = data["nbre d'heures"].apply(lambda x: pd.to_timedelta(x))
data["date_fin"] = data.apply(lambda x: x["date_debut"] + x["time_delta"], axis=1)
data.head()

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,règlement,montant HT,Texte,date_debut,time_delta,date_fin
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00


In [78]:
data["marge"] = data.apply(lambda x: x["total HT"] - x["montant HT"], axis=1)

In [79]:
data["Propriété"] = data.apply(lambda x: x["hôtel"].split(" (")[0] if "www" in x["Propriété"] else x, axis=1)["Propriété"]

In [80]:
data["Propriété_clean"] = data["hôtel"].apply(lambda x: x.split(" (")[0])

In [81]:
data["periode_fin"] = data["date_fin"].dt.strftime('%m-%Y')

In [82]:
data .head(25)

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,règlement,montant HT,Texte,date_debut,time_delta,date_fin,marge,Propriété_clean,periode_fin
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00,72,Crystal,01-2024
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00,45,Milie Rose,01-2024
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00,60,Touraine,01-2024
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00,72,Crystal,01-2024
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00,45,Milie Rose,01-2024
5,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,05/01/2024 19:30 → 06/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-05 19:30:00,0 days 12:00:00,2024-01-06 07:30:00,60,Touraine,01-2024
6,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,04/01/2024 22:30 → 05/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-04 22:30:00,0 days 09:30:00,2024-01-05 08:00:00,45,Milie Rose,01-2024
7,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Eric Da Silva (https://www.notion.so/Eric-Da-S...,04/01/2024 19:30 → 05/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-04 19:30:00,0 days 12:00:00,2024-01-05 07:30:00,60,Touraine,01-2024
8,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Emma Badani (https://www.notion.so/Emma-Badani...,03/01/2024 15:30 → 22:00,NaN,NaN,20.0,6hr 30min,120,NaN,NaN,90,NaN,2024-01-03 15:30:00,0 days 06:30:00,2024-01-03 22:00:00,30,Milie Rose,01-2024
9,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Emma Badani (https://www.notion.so/Emma-Badani...,02/01/2024 15:30 → 22:00,NaN,NaN,20.0,6hr 30min,120,NaN,NaN,90,NaN,2024-01-02 15:30:00,0 days 06:30:00,2024-01-02 22:00:00,30,Milie Rose,01-2024


In [83]:
fig = go.Figure()

fig.add_trace(go.Bar(x=data["periode_fin"], y=data["marge"]))

fig.update_traces(dict(marker_line_width=0))

In [84]:
data['Annee'] = data['date_fin'].dt.year.astype(str)
data['Mois'] = data['date_fin'].dt.to_period('M').astype(str)
data['Semaine'] = data['date_fin'].dt.to_period('W-Mon').astype(str)

# Création du graphique à barres empilées

fig_annee = go.Figure()
grouped_data = data.groupby('Annee')['marge'].sum()
fig_annee.add_trace(go.Bar(x=grouped_data.index, y=grouped_data, name='Annee', marker_color='violet'))

fig_mois = go.Figure()
grouped_data = data.groupby('Mois')['marge'].sum()
fig_mois.add_trace(go.Bar(x=grouped_data.index, y=grouped_data, name='Mois', marker_color='beige'))

fig_semaine = go.Figure()
grouped_data = data.groupby('Semaine')['marge'].sum()
fig_semaine.add_trace(go.Bar(x=grouped_data.index, y=grouped_data, name='Semaine', marker_color='purple'))


fig.data[1].visible = True

# Mise en forme du graphique
for fig in [fig_annee, fig_mois, fig_semaine]:
    fig.update_layout(title='Évolution due la marge',
                  xaxis_title='Période',
                  yaxis_title='Marge',
                  barmode='stack')
    
fig_mois.show()

buttons = [dict(label="Année", method="update", args=[{"visible": [True, False, False]}]),
           dict(label="Mois", method="update", args=[{"visible": [False, True, False]}]),
           dict(label="Semaine", method="update", args=[{"visible": [False, False, True]}])]

for fig in [fig_annee, fig_mois, fig_semaine]:
    fig.update_layout(updatemenus=[dict(showactive=True, buttons=buttons, x=0.1, xanchor='left', y=1.15, yanchor='top')])

# Affichage des graphiques en fonction du bouton sélectionné
buttons_js = f'''
<script>
  var buttons = document.querySelector('.updatemenu').childNodes;
  buttons.forEach(function(b) {{
    b.addEventListener('click', function() {{
      Plotly.update({{"height": 600}}, {{"height": 600}});
    }});
  }});
</script>
'''

for fig in [fig_annee, fig_mois, fig_semaine]:
    fig.show(config={'modeBarButtonsToAdd': [buttons_js]})

IndexError: tuple index out of range

In [ ]:
df_agg = data.groupby(data['date_fin'].dt.year, data["date_fin"].dt.to_period("M"))["marge"].sum().reset_index()
        

TypeError: unhashable type: 'Series'

In [ ]:
data["Mois"] = data["date_fin"].dt.to_period("M").astype(str)

In [ ]:
data["statuts"].value_counts()

annulé     392
urgente    225
Name: statuts, dtype: int64

In [ ]:
datetime.datetime.now().replace(year=datetime.datetime.now().year - 1).strftime("%Y-%m")

'2022-11'

In [ ]:
data["Mois"] = data["date_fin"].dt.to_period("M").astype(str)

In [ ]:
data

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,...,Texte,date_debut,time_delta,date_fin,marge,Propriété_clean,periode_fin,Annee,Mois,Semaine
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,...,NaN,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00,72,Crystal,01-2024,2024,2024-01,2024-01-02/2024-01-08
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,...,NaN,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00,45,Milie Rose,01-2024,2024,2024-01,2024-01-02/2024-01-08
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,...,NaN,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00,60,Touraine,01-2024,2024,2024-01,2024-01-02/2024-01-08
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,...,NaN,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00,72,Crystal,01-2024,2024,2024-01,2024-01-02/2024-01-08
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,...,NaN,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00,45,Milie Rose,01-2024,2024,2024-01,2024-01-02/2024-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4765,Courseine,Courseine (https://www.notion.so/Courseine-08e...,Mohamed Belbachir (https://www.notion.so/Moham...,03/12/2021 14:30 → 19:30,NaN,NaN,20.0,5hr 0min,100,NaN,...,NaN,2021-12-03 14:30:00,0 days 05:00:00,2021-12-03 19:30:00,25,Courseine,12-2021,2021,2021-12,2021-11-30/2021-12-06
4766,Prélude,Prélude (https://www.notion.so/Pr-lude-8f471c1...,Valentin Tempez (https://www.notion.so/Valenti...,03/12/2021 7:00 → 15:00,NaN,NaN,20.0,8hr 0min,160,NaN,...,NaN,2021-12-03 07:00:00,0 days 08:00:00,2021-12-03 15:00:00,40,Prélude,12-2021,2021,2021-12,2021-11-30/2021-12-06
4767,Tsuba,Tsuba (https://www.notion.so/Tsuba-6ee550393da...,Yves Constant (https://www.notion.so/Yves-Cons...,02/12/2021 22:30 → 03/12/2021 7:15,NaN,NaN,20.0,8hr 45min,160,NaN,...,NaN,2021-12-02 22:30:00,0 days 08:45:00,2021-12-03 07:15:00,40,Tsuba,12-2021,2021,2021-12,2021-11-30/2021-12-06
4768,Prélude,Prélude (https://www.notion.so/Pr-lude-8f471c1...,Sorel Pambou (https://www.notion.so/Sorel-Pamb...,01/12/2021 12:30 → 21:00,NaN,NaN,20.0,8hr 30min,160,NaN,...,NaN,2021-12-01 12:30:00,0 days 08:30:00,2021-12-01 21:00:00,40,Prélude,12-2021,2021,2021-12,2021-11-30/2021-12-06


In [ ]:
sum(data[data['Mois']==month]["total HT"])

39317

In [ ]:
(3/7).format()

AttributeError: 'float' object has no attribute 'format'

In [ ]:
max(data.groupby('Mois')['marge'])


('2024-01',
 0      72
 1      45
 2      60
 3      72
 4      45
 5      60
 6      45
 7      60
 8      30
 9      30
 10    100
 11    120
 12    120
 13    120
 Name: marge, dtype: int64)

In [90]:
max(data['date_fin']).strftime("%B %Y")

'January 2024'

In [91]:
data["Jour"] = data["date_fin"].dt.to_period("D").astype(str)

In [92]:
DATA

NameError: name 'DATA' is not defined

In [93]:
data

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,...,date_debut,time_delta,date_fin,marge,Propriété_clean,periode_fin,Annee,Mois,Semaine,Jour
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,...,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00,72,Crystal,01-2024,2024,2024-01,2024-01-02/2024-01-08,2024-01-07
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,...,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00,45,Milie Rose,01-2024,2024,2024-01,2024-01-02/2024-01-08,2024-01-07
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,...,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00,60,Touraine,01-2024,2024,2024-01,2024-01-02/2024-01-08,2024-01-07
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,...,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00,72,Crystal,01-2024,2024,2024-01,2024-01-02/2024-01-08,2024-01-06
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,...,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00,45,Milie Rose,01-2024,2024,2024-01,2024-01-02/2024-01-08,2024-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4765,Courseine,Courseine (https://www.notion.so/Courseine-08e...,Mohamed Belbachir (https://www.notion.so/Moham...,03/12/2021 14:30 → 19:30,NaN,NaN,20.0,5hr 0min,100,NaN,...,2021-12-03 14:30:00,0 days 05:00:00,2021-12-03 19:30:00,25,Courseine,12-2021,2021,2021-12,2021-11-30/2021-12-06,2021-12-03
4766,Prélude,Prélude (https://www.notion.so/Pr-lude-8f471c1...,Valentin Tempez (https://www.notion.so/Valenti...,03/12/2021 7:00 → 15:00,NaN,NaN,20.0,8hr 0min,160,NaN,...,2021-12-03 07:00:00,0 days 08:00:00,2021-12-03 15:00:00,40,Prélude,12-2021,2021,2021-12,2021-11-30/2021-12-06,2021-12-03
4767,Tsuba,Tsuba (https://www.notion.so/Tsuba-6ee550393da...,Yves Constant (https://www.notion.so/Yves-Cons...,02/12/2021 22:30 → 03/12/2021 7:15,NaN,NaN,20.0,8hr 45min,160,NaN,...,2021-12-02 22:30:00,0 days 08:45:00,2021-12-03 07:15:00,40,Tsuba,12-2021,2021,2021-12,2021-11-30/2021-12-06,2021-12-03
4768,Prélude,Prélude (https://www.notion.so/Pr-lude-8f471c1...,Sorel Pambou (https://www.notion.so/Sorel-Pamb...,01/12/2021 12:30 → 21:00,NaN,NaN,20.0,8hr 30min,160,NaN,...,2021-12-01 12:30:00,0 days 08:30:00,2021-12-01 21:00:00,40,Prélude,12-2021,2021,2021-12,2021-11-30/2021-12-06,2021-12-01


In [95]:
data = data[data["Annee"] == datetime.datetime.now().replace(year=datetime.datetime.now().year - 1).strftime("%Y")]

In [96]:
data

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,...,date_debut,time_delta,date_fin,marge,Propriété_clean,periode_fin,Annee,Mois,Semaine,Jour
2851,Imprimerie,Imprimerie (https://www.notion.so/Imprimerie-3...,Stanley Chery (https://www.notion.so/Stanley-C...,31/12/2022 16:00 → 22:00,urgente,25.0,20.0,6hr 0min,150,envoyée,...,2022-12-31 16:00:00,0 days 06:00:00,2022-12-31 22:00:00,60,Imprimerie,12-2022,2022,2022-12,2022-12-27/2023-01-02,2022-12-31
2852,Ibis Montparnasse,Ibis Montparnasse (https://www.notion.so/Ibis-...,Léopold Mendy (https://www.notion.so/L-opold-M...,31/12/2022 15:00 → 21:00,urgente,25.0,20.0,6hr 0min,150,envoyée,...,2022-12-31 15:00:00,0 days 06:00:00,2022-12-31 21:00:00,60,Ibis Montparnasse,12-2022,2022,2022-12,2022-12-27/2023-01-02,2022-12-31
2853,Ibis Montparnasse,Ibis Montparnasse (https://www.notion.so/Ibis-...,Serine Adili (https://www.notion.so/Serine-Ad...,31/12/2022 14:45 → 21:45,urgente,25.0,20.0,7hr 0min,175,envoyée,...,2022-12-31 14:45:00,0 days 07:00:00,2022-12-31 21:45:00,70,Ibis Montparnasse,12-2022,2022,2022-12,2022-12-27/2023-01-02,2022-12-31
2854,Ibis Montparnasse,Ibis Montparnasse (https://www.notion.so/Ibis-...,Shaines Ait Lhaj (https://www.notion.so/Shaine...,31/12/2022 8:00 → 14:00,urgente,25.0,20.0,6hr 0min,150,envoyée,...,2022-12-31 08:00:00,0 days 06:00:00,2022-12-31 14:00:00,60,Ibis Montparnasse,12-2022,2022,2022-12,2022-12-27/2023-01-02,2022-12-31
2855,Le Grand Quartier,Le Grand Quartier (https://www.notion.so/Le-Gr...,Balikissou Akinocho (https://www.notion.so/Bal...,31/12/2022 6:00 → 12:00,urgente,25.0,20.0,6hr 0min,150,envoyée,...,2022-12-31 06:00:00,0 days 06:00:00,2022-12-31 12:00:00,60,Le Grand Quartier,12-2022,2022,2022-12,2022-12-27/2023-01-02,2022-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4684,Victoire et Germain,Victoire et Germain (https://www.notion.so/Vic...,Jérôme Agbo (https://www.notion.so/J-r-me-Agbo...,01/01/2022 20:00 → 02/01/2022 8:00,NaN,NaN,20.0,12hr 0min,240,NaN,...,2022-01-01 20:00:00,0 days 12:00:00,2022-01-02 08:00:00,60,Victoire et Germain,01-2022,2022,2022-01,2021-12-28/2022-01-03,2022-01-02
4685,Prélude,Prélude (https://www.notion.so/Pr-lude-8f471c1...,Fanny Vicente (https://www.notion.so/Fanny-Vic...,01/01/2022 12:00 → 21:00,NaN,NaN,20.0,9hr 0min,180,NaN,...,2022-01-01 12:00:00,0 days 09:00:00,2022-01-01 21:00:00,45,Prélude,01-2022,2022,2022-01,2021-12-28/2022-01-03,2022-01-01
4686,Imprimerie,Imprimerie (https://www.notion.so/Imprimerie-3...,Alexandra Patel (https://www.notion.so/Alexand...,01/01/2022 8:00 → 20:00,NaN,NaN,20.0,12hr 0min,240,NaN,...,2022-01-01 08:00:00,0 days 12:00:00,2022-01-01 20:00:00,60,Imprimerie,01-2022,2022,2022-01,2021-12-28/2022-01-03,2022-01-01
4687,Victoire et Germain,Victoire et Germain (https://www.notion.so/Vic...,Jérôme Agbo (https://www.notion.so/J-r-me-Agbo...,01/01/2022 8:00 → 14:00,annulé,NaN,20.0,6hr 0min,120,NaN,...,2022-01-01 08:00:00,0 days 06:00:00,2022-01-01 14:00:00,30,Victoire et Germain,01-2022,2022,2022-01,2021-12-28/2022-01-03,2022-01-01
